### Sequence to Sequence 
- the translation will contain a different number of words than the input  

![](https://isaacchanghau.github.io/img/nlp/seq2seq-neuralconver/seq2seq.png)

### Encoder
- No outputs because not making predictions
- Only keep final state \begin{equation*}h_t \end{equation*}
- return_sequence = False in Keras


### Decoder
- New RNN unit with its own weights
- Must have the same vector size(M) 
- Pass start of sentence token into the X input

\begin{equation*}
y_1\ becames\ x_2 \\
y_2\ becames\ x_3 \\
\end{equation*}

- Dense layer comes after the LSTM with output size V
- Language modelling 

\begin{equation*}
P(w_t | w_{t-1},w_{t-2},...)
\end{equation*}

### Summary
- Solve the problem of mapping 
\begin{equation*}
an\ input\ of\ length\ T_x \ != \ output\ length\ T_y\\
\end{equation*}

### What tasks look like machine translation?
- Story and Question concatenated to form input sequence
- Input sequence is transformed by encoder into a thought vector
- Thought vector is decoded to form an answer

### Chatbots
- I personally don't think Seq2Seq is well suited to Chatbots
- Real conversations involve an idea spanning multiple back and forth statements by the agents involved can even return to a previous topic much later
- conversation with only binary question-answer pairs is awkward
- Seq2Seq only learns to memorize the request-response pairs


### Teacher Forcing
- Pass in the true target sequence into the bottom of the decoder RNN


### Keras
- Keras must have constant-sized inputs
- Decoder input length during training is Ty
- Decoder input length during prediction is 1

#### Keras impl
- create 2 different models

emb = Embedding()  
lstm = LSTM()  
dense = Dense()  
  
input1 = Input(lenght = Ty)  
model1  = Model(input1, dense(lstm(emb(input1))))  
  
input2 = Input(lenght = 1)  
model2  = Model(input1, dense(lstm(emb(input2))))  
  
h = encoder model output x = SOS  
for t in range(Ty)  
    x,h = model2.predict(x,h)  
    




#### what should this remind you of?

- In seq2seq, $h_{0}^{'}$ is the encoder's output
- Here it is just 0

#### Poetry generation

- it is the same as the decoder in seq2seq

### Code(Poetry)

In [1]:
import os
import sys
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.models import Model
from keras.layers import Dense, Embedding, Input, LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam, SGD

import keras.backend as K
if len(K.tensorflow_backend._get_available_gpus()) > 0:
    from keras.layers import CuDNNLSTM as LSTM
    from keras.layers import CuDNNGRU as GRU



Using TensorFlow backend.


In [2]:
# some configuration
MAX_SEQUENCE_LENGTH = 100
MAX_VOCAB_SIZE = 3000
EMBEDDING_DIM = 50
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 128
EPOCHS = 2000
LATENT_DIM = 25


In [3]:
count = 0

# load in the data
input_texts = []
target_texts = []
for line in open('./large_files/robert_frost.txt'):
    line = line.rstrip()
    if not line:
        continue

    input_line = '<sos> ' + line
    target_line = line + ' <eos>'

    input_texts.append(input_line)
    target_texts.append(target_line)
    
    if count > 10:
        break
    count += 1
    
all_lines = input_texts + target_texts


In [4]:
input_texts

['<sos> Two roads diverged in a yellow wood,',
 '<sos> And sorry I could not travel both',
 '<sos> And be one traveler, long I stood',
 '<sos> And looked down one as far as I could',
 '<sos> To where it bent in the undergrowth;',
 '<sos> Then took the other, as just as fair,',
 '<sos> And having perhaps the better claim',
 '<sos> Because it was grassy and wanted wear,',
 '<sos> Though as for that the passing there',
 '<sos> Had worn them really about the same,',
 '<sos> And both that morning equally lay',
 '<sos> In leaves no step had trodden black.']

In [5]:
# convert the sentences (strings) into integers
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, filters='')
tokenizer.fit_on_texts(all_lines)
input_sequences = tokenizer.texts_to_sequences(input_texts)
target_sequences = tokenizer.texts_to_sequences(target_texts)


In [6]:
len(input_sequences),input_sequences[0]

(12, [1, 14, 15, 16, 6, 17, 18, 19])

In [7]:
# find max seq length
max_sequence_length_from_data = max(len(s) for s in input_sequences)
print('Max sequence length:', max_sequence_length_from_data)


# get word -> integer mapping
word2idx = tokenizer.word_index
print('Found %s unique tokens.' % len(word2idx))


Max sequence length: 10
Found 64 unique tokens.


In [8]:
word2idx

{'<eos>': 3,
 '<sos>': 1,
 'a': 17,
 'about': 55,
 'and': 2,
 'as': 4,
 'be': 23,
 'because': 43,
 'bent': 32,
 'better': 41,
 'black.': 64,
 'both': 9,
 'claim': 42,
 'could': 8,
 'diverged': 16,
 'down': 28,
 'equally': 58,
 'fair,': 38,
 'far': 29,
 'for': 49,
 'grassy': 45,
 'had': 13,
 'having': 39,
 'i': 7,
 'in': 6,
 'it': 11,
 'just': 37,
 'lay': 59,
 'leaves': 60,
 'long': 25,
 'looked': 27,
 'morning': 57,
 'no': 61,
 'not': 21,
 'one': 10,
 'other,': 36,
 'passing': 50,
 'perhaps': 40,
 'really': 54,
 'roads': 15,
 'same,': 56,
 'sorry': 20,
 'step': 62,
 'stood': 26,
 'that': 12,
 'the': 5,
 'them': 53,
 'then': 34,
 'there': 51,
 'though': 48,
 'to': 30,
 'took': 35,
 'travel': 22,
 'traveler,': 24,
 'trodden': 63,
 'two': 14,
 'undergrowth;': 33,
 'wanted': 46,
 'was': 44,
 'wear,': 47,
 'where': 31,
 'wood,': 19,
 'worn': 52,
 'yellow': 18}

In [9]:
# pad sequences so that we get a N x T matrix
max_sequence_length = min(max_sequence_length_from_data, MAX_SEQUENCE_LENGTH)
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='post')
target_sequences = pad_sequences(target_sequences, maxlen=max_sequence_length, padding='post')
print('Shape of data tensor:', input_sequences.shape)


Shape of data tensor: (12, 10)


In [10]:
# load in pre-trained word vectors
print('Loading word vectors...')
word2vec = {}
count = 0
with open(os.path.join('./large_files/glove.6B/glove.6B.%sd.txt' % EMBEDDING_DIM)) as f:
  # is just a space-separated text file in the format:
  # word vec[0] vec[1] vec[2] ...
    for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec
        
        if count > 10:
            break
        count += 1
print('Found %s word vectors.' % len(word2vec))



Loading word vectors...
Found 12 word vectors.


In [11]:
# prepare embedding matrix
print('Filling pre-trained embeddings...')
num_words = min(MAX_VOCAB_SIZE, len(word2idx) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word2idx.items():
    if i < MAX_VOCAB_SIZE:
        embedding_vector = word2vec.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all zeros.
            embedding_matrix[i] = embedding_vector


Filling pre-trained embeddings...


In [12]:
num_words = min(MAX_VOCAB_SIZE, len(word2idx) + 1)
# one-hot the targets (can't use sparse cross-entropy)
one_hot_targets = np.zeros((len(input_sequences), max_sequence_length, num_words))
print("one_hot_targets {}".format(one_hot_targets.shape))
for i, target_sequence in enumerate(target_sequences):
    for t, word in enumerate(target_sequence):
        if word > 0:
            one_hot_targets[i, t, word] = 1



one_hot_targets (12, 10, 65)


In [13]:
target_sequences

array([[14, 15, 16,  6, 17, 18, 19,  3,  0,  0],
       [ 2, 20,  7,  8, 21, 22,  9,  3,  0,  0],
       [ 2, 23, 10, 24, 25,  7, 26,  3,  0,  0],
       [ 2, 27, 28, 10,  4, 29,  4,  7,  8,  3],
       [30, 31, 11, 32,  6,  5, 33,  3,  0,  0],
       [34, 35,  5, 36,  4, 37,  4, 38,  3,  0],
       [ 2, 39, 40,  5, 41, 42,  3,  0,  0,  0],
       [43, 11, 44, 45,  2, 46, 47,  3,  0,  0],
       [48,  4, 49, 12,  5, 50, 51,  3,  0,  0],
       [13, 52, 53, 54, 55,  5, 56,  3,  0,  0],
       [ 2,  9, 12, 57, 58, 59,  3,  0,  0,  0],
       [ 6, 60, 61, 62, 13, 63, 64,  3,  0,  0]], dtype=int32)

In [14]:
# load pre-trained word embeddings into an Embedding layer
embedding_layer = Embedding(
  num_words,
  EMBEDDING_DIM,
  weights=[embedding_matrix],
  # trainable=False
)



In [15]:

print('Building model...')

# create an LSTM network with a single LSTM
input_ = Input(shape=(max_sequence_length,))

initial_h = Input(shape=(LATENT_DIM,))
initial_c = Input(shape=(LATENT_DIM,))

print(input_,initial_h,initial_c)
x = embedding_layer(input_)
lstm = LSTM(LATENT_DIM, return_sequences=True, return_state=True)
x, _, _ = lstm(x, initial_state=[initial_h, initial_c]) # don't need the states here
dense = Dense(num_words, activation='softmax')
output = dense(x)

model = Model([input_, initial_h, initial_c], output)
model.compile(
  loss='categorical_crossentropy',
  # optimizer='rmsprop',
  optimizer=Adam(lr=0.01),
  # optimizer=SGD(lr=0.01, momentum=0.9),
  metrics=['accuracy']
)


Building model...
Tensor("input_1:0", shape=(?, 10), dtype=float32) Tensor("input_2:0", shape=(?, 25), dtype=float32) Tensor("input_3:0", shape=(?, 25), dtype=float32)


In [16]:
max_sequence_length,output

(10, <tf.Tensor 'dense_1/truediv:0' shape=(?, 10, 65) dtype=float32>)

In [17]:
print('Training model...')
z = np.zeros((len(input_sequences), LATENT_DIM))
print("z {}".format(z.shape))
r = model.fit(
  [input_sequences, z, z],
  one_hot_targets,
  batch_size=BATCH_SIZE,
  epochs=EPOCHS,
  validation_split=VALIDATION_SPLIT
)


Training model...
z (12, 25)
Train on 9 samples, validate on 3 samples
Epoch 1/2000
9/9 [==============================] - 2s 194ms/step - loss: 3.4355 - acc: 0.1778 - val_loss: 3.2121 - val_acc: 0.0667
Epoch 2/2000
9/9 [==============================] - 0s 3ms/step - loss: 3.3977 - acc: 0.0889 - val_loss: 3.2122 - val_acc: 0.1000
Epoch 3/2000
9/9 [==============================] - 0s 2ms/step - loss: 3.3649 - acc: 0.1778 - val_loss: 3.2132 - val_acc: 0.1333
Epoch 4/2000
9/9 [==============================] - 0s 4ms/step - loss: 3.3224 - acc: 0.1889 - val_loss: 3.2152 - val_acc: 0.1333
Epoch 5/2000
9/9 [==============================] - 0s 4ms/step - loss: 3.2761 - acc: 0.1889 - val_loss: 3.2192 - val_acc: 0.1333
Epoch 6/2000
9/9 [==============================] - 0s 3ms/step - loss: 3.2178 - acc: 0.1667 - val_loss: 3.2265 - val_acc: 0.1333
Epoch 7/2000
9/9 [==============================] - 0s 3ms/step - loss: 3.1494 - acc: 0.1556 - val_loss: 3.2398 - val_acc: 0.1333
Epoch 8/2000
9/9 

9/9 [==============================] - 0s 3ms/step - loss: 0.3559 - acc: 0.7333 - val_loss: 5.7553 - val_acc: 0.1000
Epoch 64/2000
9/9 [==============================] - 0s 4ms/step - loss: 0.3476 - acc: 0.7333 - val_loss: 5.7714 - val_acc: 0.1000
Epoch 65/2000
9/9 [==============================] - 0s 3ms/step - loss: 0.3398 - acc: 0.7333 - val_loss: 5.7880 - val_acc: 0.1000
Epoch 66/2000
9/9 [==============================] - 0s 4ms/step - loss: 0.3326 - acc: 0.7333 - val_loss: 5.8054 - val_acc: 0.1000
Epoch 67/2000
9/9 [==============================] - 0s 4ms/step - loss: 0.3260 - acc: 0.7333 - val_loss: 5.8223 - val_acc: 0.1000
Epoch 68/2000
9/9 [==============================] - 0s 2ms/step - loss: 0.3201 - acc: 0.7333 - val_loss: 5.8382 - val_acc: 0.1000
Epoch 69/2000
9/9 [==============================] - 0s 4ms/step - loss: 0.3147 - acc: 0.7333 - val_loss: 5.8526 - val_acc: 0.1000
Epoch 70/2000
9/9 [==============================] - 0s 2ms/step - loss: 0.3097 - acc: 0.7333 - v

Epoch 126/2000
9/9 [==============================] - 0s 3ms/step - loss: 0.2361 - acc: 0.7333 - val_loss: 6.1747 - val_acc: 0.1000
Epoch 127/2000
9/9 [==============================] - 0s 3ms/step - loss: 0.2359 - acc: 0.7333 - val_loss: 6.1775 - val_acc: 0.1000
Epoch 128/2000
9/9 [==============================] - 0s 3ms/step - loss: 0.2356 - acc: 0.7333 - val_loss: 6.1804 - val_acc: 0.1000
Epoch 129/2000
9/9 [==============================] - 0s 2ms/step - loss: 0.2354 - acc: 0.7333 - val_loss: 6.1834 - val_acc: 0.1000
Epoch 130/2000
9/9 [==============================] - 0s 2ms/step - loss: 0.2351 - acc: 0.7333 - val_loss: 6.1866 - val_acc: 0.1000
Epoch 131/2000
9/9 [==============================] - 0s 4ms/step - loss: 0.2349 - acc: 0.7333 - val_loss: 6.1899 - val_acc: 0.1000
Epoch 132/2000
9/9 [==============================] - 0s 3ms/step - loss: 0.2347 - acc: 0.7333 - val_loss: 6.1931 - val_acc: 0.1000
Epoch 133/2000
9/9 [==============================] - 0s 3ms/step - loss: 0.

9/9 [==============================] - 0s 2ms/step - loss: 0.2277 - acc: 0.7333 - val_loss: 6.3389 - val_acc: 0.1000
Epoch 189/2000
9/9 [==============================] - 0s 4ms/step - loss: 0.2276 - acc: 0.7333 - val_loss: 6.3411 - val_acc: 0.1000
Epoch 190/2000
9/9 [==============================] - 0s 4ms/step - loss: 0.2276 - acc: 0.7333 - val_loss: 6.3434 - val_acc: 0.1000
Epoch 191/2000
9/9 [==============================] - 0s 3ms/step - loss: 0.2275 - acc: 0.7333 - val_loss: 6.3456 - val_acc: 0.1000
Epoch 192/2000
9/9 [==============================] - 0s 3ms/step - loss: 0.2274 - acc: 0.7333 - val_loss: 6.3478 - val_acc: 0.1000
Epoch 193/2000
9/9 [==============================] - 0s 3ms/step - loss: 0.2274 - acc: 0.7333 - val_loss: 6.3500 - val_acc: 0.1000
Epoch 194/2000
9/9 [==============================] - 0s 2ms/step - loss: 0.2273 - acc: 0.7333 - val_loss: 6.3524 - val_acc: 0.1000
Epoch 195/2000
9/9 [==============================] - 0s 4ms/step - loss: 0.2272 - acc: 0.7

Epoch 251/2000
9/9 [==============================] - 0s 3ms/step - loss: 0.2247 - acc: 0.7333 - val_loss: 6.4767 - val_acc: 0.1000
Epoch 252/2000
9/9 [==============================] - 0s 4ms/step - loss: 0.2246 - acc: 0.7333 - val_loss: 6.4786 - val_acc: 0.1000
Epoch 253/2000
9/9 [==============================] - 0s 2ms/step - loss: 0.2246 - acc: 0.7333 - val_loss: 6.4805 - val_acc: 0.1000
Epoch 254/2000
9/9 [==============================] - 0s 2ms/step - loss: 0.2246 - acc: 0.7333 - val_loss: 6.4823 - val_acc: 0.1000
Epoch 255/2000
9/9 [==============================] - 0s 1ms/step - loss: 0.2245 - acc: 0.7333 - val_loss: 6.4842 - val_acc: 0.1000
Epoch 256/2000
9/9 [==============================] - 0s 3ms/step - loss: 0.2245 - acc: 0.7333 - val_loss: 6.4860 - val_acc: 0.1000
Epoch 257/2000
9/9 [==============================] - 0s 2ms/step - loss: 0.2245 - acc: 0.7333 - val_loss: 6.4878 - val_acc: 0.1000
Epoch 258/2000
9/9 [==============================] - 0s 2ms/step - loss: 0.

...

Epoch 1745/2000
9/9 [==============================] - 0s 3ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.4976 - val_acc: 0.0667
Epoch 1746/2000
9/9 [==============================] - 0s 2ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.4979 - val_acc: 0.0667
Epoch 1747/2000
9/9 [==============================] - 0s 3ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.4982 - val_acc: 0.0667
Epoch 1748/2000
9/9 [==============================] - 0s 2ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.4985 - val_acc: 0.0667
Epoch 1749/2000
9/9 [==============================] - 0s 3ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.4988 - val_acc: 0.0667
Epoch 1750/2000
9/9 [==============================] - 0s 4ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.4991 - val_acc: 0.0667
Epoch 1751/2000
9/9 [==============================] - 0s 2ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.4994 - val_acc: 0.0667
Epoch 1752/2000
9/9 [==============================] - 0s 4ms/step - 

Epoch 1807/2000
9/9 [==============================] - 0s 4ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.5160 - val_acc: 0.0667
Epoch 1808/2000
9/9 [==============================] - 0s 4ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.5163 - val_acc: 0.0667
Epoch 1809/2000
9/9 [==============================] - 0s 3ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.5166 - val_acc: 0.0667
Epoch 1810/2000
9/9 [==============================] - 0s 3ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.5169 - val_acc: 0.0667
Epoch 1811/2000
9/9 [==============================] - 0s 3ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.5172 - val_acc: 0.0667
Epoch 1812/2000
9/9 [==============================] - 0s 2ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.5175 - val_acc: 0.0667
Epoch 1813/2000
9/9 [==============================] - 0s 2ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.5178 - val_acc: 0.0667
Epoch 1814/2000
9/9 [==============================] - 0s 3ms/step - 

Epoch 1869/2000
9/9 [==============================] - 0s 2ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.5341 - val_acc: 0.0667
Epoch 1870/2000
9/9 [==============================] - 0s 3ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.5344 - val_acc: 0.0667
Epoch 1871/2000
9/9 [==============================] - 0s 3ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.5346 - val_acc: 0.0667
Epoch 1872/2000
9/9 [==============================] - 0s 2ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.5349 - val_acc: 0.0667
Epoch 1873/2000
9/9 [==============================] - 0s 3ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.5352 - val_acc: 0.0667
Epoch 1874/2000
9/9 [==============================] - 0s 2ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.5355 - val_acc: 0.0667
Epoch 1875/2000
9/9 [==============================] - 0s 3ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.5358 - val_acc: 0.0667
Epoch 1876/2000
9/9 [==============================] - 0s 2ms/step - 

Epoch 1931/2000
9/9 [==============================] - 0s 3ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.5517 - val_acc: 0.0667
Epoch 1932/2000
9/9 [==============================] - 0s 3ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.5520 - val_acc: 0.0667
Epoch 1933/2000
9/9 [==============================] - 0s 4ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.5523 - val_acc: 0.0667
Epoch 1934/2000
9/9 [==============================] - 0s 3ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.5526 - val_acc: 0.0667
Epoch 1935/2000
9/9 [==============================] - 0s 3ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.5529 - val_acc: 0.0667
Epoch 1936/2000
9/9 [==============================] - 0s 3ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.5531 - val_acc: 0.0667
Epoch 1937/2000
9/9 [==============================] - 0s 3ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.5534 - val_acc: 0.0667
Epoch 1938/2000
9/9 [==============================] - 0s 2ms/step - 

Epoch 1993/2000
9/9 [==============================] - 0s 2ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.5693 - val_acc: 0.0667
Epoch 1994/2000
9/9 [==============================] - 0s 3ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.5696 - val_acc: 0.0667
Epoch 1995/2000
9/9 [==============================] - 0s 2ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.5699 - val_acc: 0.0667
Epoch 1996/2000
9/9 [==============================] - 0s 4ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.5702 - val_acc: 0.0667
Epoch 1997/2000
9/9 [==============================] - 0s 3ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.5704 - val_acc: 0.0667
Epoch 1998/2000
9/9 [==============================] - 0s 3ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.5707 - val_acc: 0.0667
Epoch 1999/2000
9/9 [==============================] - 0s 2ms/step - loss: 0.2199 - acc: 0.7333 - val_loss: 7.5710 - val_acc: 0.0667
Epoch 2000/2000
9/9 [==============================] - 0s 3ms/step - 

In [28]:
input_sequences.shape

(12, 10)

In [31]:
# make a sampling model
input2 = Input(shape=(1,)) # we'll only input one word at a time
x = embedding_layer(input2)
# lstm2 = LSTM(LATENT_DIM, return_sequences=True, return_state=True)
x, h, c = lstm(x, initial_state=[initial_h, initial_c]) # now we need states to feed back in
output2 = dense(x)
sampling_model = Model([input2, initial_h, initial_c], [output2, h, c])


- ## prediction lstm and training lstm model weights are the same 

In [32]:
sampling_model.weights,model.weights

([<tf.Variable 'embedding_1/embeddings:0' shape=(65, 50) dtype=float32_ref>,
  <tf.Variable 'lstm_1/kernel:0' shape=(50, 100) dtype=float32_ref>,
  <tf.Variable 'lstm_1/recurrent_kernel:0' shape=(25, 100) dtype=float32_ref>,
  <tf.Variable 'lstm_1/bias:0' shape=(100,) dtype=float32_ref>,
  <tf.Variable 'dense_1/kernel:0' shape=(25, 65) dtype=float32_ref>,
  <tf.Variable 'dense_1/bias:0' shape=(65,) dtype=float32_ref>],
 [<tf.Variable 'embedding_1/embeddings:0' shape=(65, 50) dtype=float32_ref>,
  <tf.Variable 'lstm_1/kernel:0' shape=(50, 100) dtype=float32_ref>,
  <tf.Variable 'lstm_1/recurrent_kernel:0' shape=(25, 100) dtype=float32_ref>,
  <tf.Variable 'lstm_1/bias:0' shape=(100,) dtype=float32_ref>,
  <tf.Variable 'dense_1/kernel:0' shape=(25, 65) dtype=float32_ref>,
  <tf.Variable 'dense_1/bias:0' shape=(65,) dtype=float32_ref>])

In [20]:
x,num_words, EMBEDDING_DIM,output2,output2[0,0]

(<tf.Tensor 'lstm_1_1/transpose_1:0' shape=(?, ?, 25) dtype=float32>,
 65,
 50,
 <tf.Tensor 'dense_1_1/truediv:0' shape=(?, 1, 65) dtype=float32>,
 <tf.Tensor 'strided_slice:0' shape=(65,) dtype=float32>)

In [33]:
# reverse word2idx dictionary to get back words
# during prediction
idx2word = {v:k for k, v in word2idx.items()}


def sample_line():
    # initial inputs
    np_input = np.array([[ word2idx['<sos>'] ]])
    h = np.zeros((1, LATENT_DIM))
    c = np.zeros((1, LATENT_DIM))

    # so we know when to quit
    eos = word2idx['<eos>']

    # store the output here
    output_sentence = []

    for _ in range(max_sequence_length):
        o, h, c = sampling_model.predict([np_input, h, c])

        # print("o.shape:", o.shape, o[0,0,:10])
        # idx = np.argmax(o[0,0])
        probs = o[0,0]
        if np.argmax(probs) == 0:
            print("wtf")
        probs[0] = 0
        probs /= probs.sum()
        idx = np.random.choice(len(probs), p=probs)
        if idx == eos:
            break

        # accuulate output
        output_sentence.append(idx2word.get(idx, '<WTF %s>' % idx))

        # make the next input into model
        np_input[0,0] = idx

    return ' '.join(output_sentence)

# generate a 4 line poem
while True:
    for _ in range(4):
        print(sample_line())

    ans = input("---generate another? [Y/n]---")
    if ans and ans[0].lower().startswith('n'):
        break


because it was grassy and wanted wear,
because it was grassy and wanted wear,
and be one traveler, long i stood
and be one traveler, long i stood
---generate another? [Y/n]---y
though as for that the passing there
though as for that the passing there
and looked down one as far as i could
then took the other, as just as fair,
---generate another? [Y/n]---n


In [46]:
np_input = np.array([[ word2idx['<sos>'] ]])
np_input.shape,np_input,np_input[0,0]

((1, 1), array([[1]]), 1)